In [2]:
import numpy as np
import pandas as pd
import pymongo
import datetime
client = pymongo.MongoClient()
db1 = client['Index']
twa = pd.DataFrame(db1['TWA00'].find({'日期': {'$gte': datetime.datetime(2008, 1, 1)}}, {'_id': 0})).set_index('日期')
vix = pd.DataFrame(db1['VIX'].find({'日期': {'$gte': datetime.datetime(2008, 1, 1)}}, {'_id': 0})).set_index('日期')
how_many_day = 1 # 隔幾天的漲跌
today_close = 14476
rate_ = []
std_ = []
pf_ = []
for d in range(1, 10):
    jump = (twa['收盤價'].shift(how_many_day) / twa['收盤價'].shift(how_many_day+d))-1
    rate = today_close / twa['收盤價'].iloc[-d] - 1
    # if rate >=0:
    idx = jump[(jump >= rate-0.002) & (jump < rate+0.002)].index
    # else:
    #     idx = jump[jump < rate].index
    next_day = (twa['收盤價'] / twa['收盤價'].shift(how_many_day) - 1).loc[idx]
    rate_.append((next_day>0).sum()/next_day.count()*100)
    std_.append(next_day.std()*100)
    pf_.append(next_day.mean()*100)
    print('近 {} 日 漲跌 {:.2f}% 後 隔{}日漲跌幅平均: {:.2f}% 上漲機率: {:.2f}% 標準差: {:.2f}%'.format(d, rate*100, how_many_day, next_day.mean()*100, (next_day>0).sum()/next_day.count()*100, next_day.std()*100))
print('平均 {} 日後漲跌 {:.2f}% 平均上漲機率 {:.2f}% 平均標準差 {:.2f}%'.format(how_many_day, np.mean(pf_), np.mean(rate_), np.mean(std_)))

近 1 日 漲跌 -0.20% 後 隔1日漲跌幅平均: -0.06% 上漲機率: 50.15% 標準差: 1.07%
近 2 日 漲跌 -0.41% 後 隔1日漲跌幅平均: 0.08% 上漲機率: 56.28% 標準差: 1.09%
近 3 日 漲跌 -0.42% 後 隔1日漲跌幅平均: 0.00% 上漲機率: 52.06% 標準差: 0.97%
近 4 日 漲跌 -0.48% 後 隔1日漲跌幅平均: 0.07% 上漲機率: 53.85% 標準差: 1.12%
近 5 日 漲跌 2.12% 後 隔1日漲跌幅平均: 0.09% 上漲機率: 57.86% 標準差: 1.04%
近 6 日 漲跌 3.34% 後 隔1日漲跌幅平均: 0.11% 上漲機率: 54.17% 標準差: 0.79%
近 7 日 漲跌 7.20% 後 隔1日漲跌幅平均: 0.82% 上漲機率: 72.73% 標準差: 1.20%
近 8 日 漲跌 6.14% 後 隔1日漲跌幅平均: 0.69% 上漲機率: 61.54% 標準差: 1.27%
近 9 日 漲跌 8.45% 後 隔1日漲跌幅平均: -0.33% 上漲機率: 57.14% 標準差: 2.51%
平均 1 日後漲跌 0.16% 平均上漲機率 57.31% 平均標準差 1.23%


In [4]:
how_many_day = 1 # 隔幾天的漲跌

rate_ = []
std_ = []
pf_ = []
twa.loc[datetime.datetime.today().strftime('%Y-%m-%d'), '收盤價'] = 14504.99
vix.loc[datetime.datetime.today().strftime('%Y-%m-%d'), '收盤價'] = 21.61
cor = vix['收盤價'].pct_change().rolling(5).corr(twa['收盤價'].pct_change())
for d in range(2, 11):
    cor = vix['收盤價'].pct_change().rolling(d).corr(twa['收盤價'].pct_change())
    a = ((twa['收盤價'] / twa['收盤價'].shift(how_many_day) - 1) > 0)
    a_ = a.sum()/a.count()
    rate = cor.iloc[-1]
    idx = cor[(cor > rate - .5)&(cor <= rate + .5)].index
    next_day = (twa['收盤價'] / twa['收盤價'].shift(how_many_day) - 1).loc[idx]
    rate_.append((next_day>0).sum()/next_day.count()*100)
    std_.append(next_day.std()*100)
    pf_.append(next_day.mean()*100)
    print('cor介於 {:.2f} ~ {:.2f} 隔{}日漲跌幅平均: {:.2f}% 上漲機率: {:.2f}% 標準差: {:.2f}% 樣本上漲機率: {:.2f}%'.format(rate-.2, rate+.2, how_many_day, next_day.mean()*100, (next_day>0).sum()/next_day.count()*100, next_day.std()*100, a_))
print('平均 {} 日後漲跌 {:.2f}% 平均上漲機率 {:.2f}% 平均標準差 {:.2f}%'.format(1, np.mean(pf_), np.mean(rate_), np.mean(std_)))

cor介於 -1.20 ~ -0.80 隔1日漲跌幅平均: -0.01% 上漲機率: 53.07% 標準差: 1.19% 樣本上漲機率: 0.54%
cor介於 -0.89 ~ -0.49 隔1日漲跌幅平均: -0.00% 上漲機率: 53.29% 標準差: 1.18% 樣本上漲機率: 0.54%
cor介於 0.68 ~ 1.08 隔1日漲跌幅平均: 0.15% 上漲機率: 57.35% 標準差: 1.25% 樣本上漲機率: 0.54%
cor介於 0.68 ~ 1.08 隔1日漲跌幅平均: 0.09% 上漲機率: 53.52% 標準差: 1.42% 樣本上漲機率: 0.54%
cor介於 0.31 ~ 0.71 隔1日漲跌幅平均: 0.09% 上漲機率: 54.91% 標準差: 1.34% 樣本上漲機率: 0.54%
cor介於 0.22 ~ 0.62 隔1日漲跌幅平均: 0.08% 上漲機率: 54.13% 標準差: 1.31% 樣本上漲機率: 0.54%
cor介於 0.21 ~ 0.61 隔1日漲跌幅平均: 0.07% 上漲機率: 52.34% 標準差: 1.31% 樣本上漲機率: 0.54%
cor介於 0.19 ~ 0.59 隔1日漲跌幅平均: 0.06% 上漲機率: 52.16% 標準差: 1.31% 樣本上漲機率: 0.54%
cor介於 0.16 ~ 0.56 隔1日漲跌幅平均: 0.05% 上漲機率: 51.30% 標準差: 1.30% 樣本上漲機率: 0.54%
平均 1 日後漲跌 0.06% 平均上漲機率 53.56% 平均標準差 1.29%


In [52]:
a = ((twa['收盤價'] / twa['收盤價'].shift(how_many_day) - 1) > 0)
a.sum()/a.count()

0.569012547735952

In [110]:
((cor > rate - .2)&(cor <= rate + .2)).sum()

164

In [114]:
idx

DatetimeIndex(['2008-01-16', '2008-01-17', '2008-01-18', '2008-01-21',
               '2008-01-22', '2008-01-23', '2008-01-24', '2008-01-25',
               '2008-01-28', '2008-02-14',
               ...
               '2021-11-16', '2021-11-17', '2021-11-18', '2021-11-19',
               '2022-01-04', '2022-01-05', '2022-11-09', '2022-11-11',
               '2022-11-14', '2022-11-15'],
              dtype='datetime64[ns]', name='日期', length=462, freq=None)